In [10]:
from selenium import webdriver
import time
import pandas as pd
import pickle
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

download_dir = '/home/arn197/CS506/Project/'
chrome_options = Options()
chrome_options.add_experimental_option('prefs',  {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    })

driver = webdriver.Chrome('./chromedriver', options = chrome_options)  # Optional argument, if not specified will search path.

## This cell was used to load the results of each alphabet and save the html page, due to memory issues when scraping directly

In [11]:
# import string
# from selenium.webdriver.support.ui import Select
# driver.set_page_load_timeout(20000)
# alpha = list(string.ascii_lowercase)
# for i in range(int(len(alpha)/2)):
#     driver.get('http://corp.sec.state.ma.us/CorpWeb/CorpSearch/CorpSearch.aspx')
#     search = driver.find_element_by_id('MainContent_txtEntityName')
#     search.send_keys(alpha[i])
#     opt = Select(driver.find_element_by_id('MainContent_ddRecordsPerPage'))
#     opt.select_by_value('All items')
#     with open(alpha[i] + '_page.html', 'w') as f:
#         f.write(driver.page_source)

## For each of the downloaded html pages, the links of all the companies were extracted and saved in a pickle file

In [13]:
# from bs4 import BeautifulSoup
# alpha = list(string.ascii_lowercase)
# for i in range(8, int(len(alpha)/2)):
#     f = open('StateSecretaryData/' + alpha[i] + '_page.html', 'r')
#     soup = BeautifulSoup(f)
#     x = soup.find(id="MainContent_SearchControl_grdSearchResultsEntity")
#     count = 0
#     print(alpha[i])
#     for i in x.find_all('a'):
#         if count < 4:
#             count += 1
#             continue
#         links.append(i.get('href'))
#     f.close()

# f = open('saved_links.pkl', 'wb')
# pickle.dump(links, f)
# f.close()

In [14]:
f = open('saved_links.pkl', 'rb')
links = pickle.load(f)
f.close()

## We take each of the saved links and search for the link to the latest annual report filing of the corresponding company

In [15]:
for link_n in range(len(links)):
    if 'http://corp.sec.state.ma.us/CorpWeb/CorpSearch/' in links[link_n]:
        driver.get(links[link_n])
    else:
        driver.get('http://corp.sec.state.ma.us/CorpWeb/CorpSearch/' + links[link_n])

    filings_btn = driver.find_element_by_id('MainContent_btnViewFilings')
    filings_btn.click()

    table = driver.find_element_by_id('MainContent_grdSearchResults')
    li = table.find_elements_by_tag_name('tr')
    ann_rep = None
    org_cert = None
    for i in li:
        if i.get_attribute('class') == 'GridHeader':
            continue
        if "Annual Report" in i.text and ann_rep == None:
            ann_rep = i.find_element_by_tag_name('a').get_attribute('href')
#     if "Certificate of Organization" in i.text and org_cert == None:
#         org_cert = i.find_element_by_tag_name('a').get_attribute('href')

## This part is still in progress - The idea is to download one pdf at a time, analyze it using a PDF library and then save the details of the companies related to healthcare

In [39]:
from PyPDF2 import PdfFileReader
from pdfminer.converter import TextConverter, PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBoxHorizontal
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

rsrcmgr = PDFResourceManager()
# Create a PDF device object.
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)

In [58]:
import os
import time
flag = 0

def check(link):
    if link == None:
        return -1
#     driver.get(ann_rep)
#     time.sleep(1)
    f = open('/home/arn197/CS506/Project/CorpSearchViewPDF.aspx', 'rb')
    pdf = PdfFileReader(f)
#     os.remove('/home/arn197/CS506/Project/CorpSearchViewPDF.aspx')
    page = pdf.getPage(0)
    text = page.extractText()
    if text == '':
        ret = -1
    else:
        for page in PDFPage.get_pages(f):
            interpreter.process_page(page)
            # receive the LTPage object for the page.
            layout = device.get_result()
            for i in layout:
                if isinstance(i, LTTextBoxHorizontal):
                    print("-----------------------------------------")
                    print(i.get_text())
        ret = 1
    f.close()
    return ret

In [59]:
ret = check(ann_rep)
if ret != -1:
    print(ret)

-----------------------------------------
MA SOC   Filing Number: 201885038250     Date: 2/22/2018 11:34:00 AM

-----------------------------------------
The Commonwealth of Massachusetts 

-----------------------------------------
Minimum Fee: $500.00 

-----------------------------------------
Secretary of the Commonwealth, Corporations Division 

-----------------------------------------
William Francis Galvin 

-----------------------------------------
One Ashburton Place, 17th floor 

-----------------------------------------
Boston, MA 02108-1512 
Telephone: (617) 727-9640 

-----------------------------------------
  

-----------------------------------------
  

-----------------------------------------
Annual Report      
(General Laws, Chapter ) 

-----------------------------------------
Identification Number:  001187544 

-----------------------------------------
Annual Report Filing Year: 2018     

-----------------------------------------
1.a. Exact name of the limited 